## Application des transformations definies dans "trasnfromers.py"

In [1]:
from transformers import *

In [2]:
import pandas as pd
from abc import ABC, abstractmethod
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pathlib import Path
from sklearn.model_selection import train_test_split

import pickle


In [4]:
columns_to_drop = [
        "piezo_station_department_name",
        "piezo_station_update_date",
        "piezo_station_commune_code_insee",
        "piezo_station_pe_label",
        "piezo_station_bdlisa_codes",
        "piezo_station_bss_code",
        "piezo_station_bss_id", 
        "piezo_bss_code",
        "piezo_measurement_date",
        "piezo_producer_name",
        "piezo_measure_nature_code",
        "meteo_name",
        "meteo_id", 
        "meteo_latitude",
        "meteo_longitude",
        "hydro_station_code",
        "hydro_method_code", 
        "hydro_method_label", 
        "hydro_qualification_label", 
        "prelev_structure_code_0",
        "prelev_structure_code_2",
        "prelev_structure_code_0",
        "prelev_commune_code_insee_0",
        "piezo_station_department_code",
        
        "meteo_DRR", 
        "meteo_temperature_min_ground", 
        "meteo_temperature_min_50cm", 
        "meteo_pressure_avg",
        "meteo_pression_maxi", 
        "meteo_wind_speed_avg_2m", 
        "meteo_wind_max_2m", 
        "meteo_wind_direction_max_inst_2m", 
        "meteo_time_wind_max_2m", 
        "meteo_wetting_duration", 
        "meteo_sunshine_duration", 
        "meteo_radiation", 
        "meteo_radiation_direct", 
        "meteo_sunshine_%", 
        "meteo_radiation_IR", 
        "meteo_radiation_UV_max", 
        "meteo_cloudiness", 
        "meteo_cloudiness_height", 
        "meteo_if_snow", 
        "meteo_if_fog", 
        "meteo_if_thunderstorm", 
        "meteo_if_sleet", 
        "meteo_if_hail", 
        "meteo_if_dew", 
        "meteo_if_black_ice", 
        "meteo_if_snow_ground", 
        "meteo_if_frost", 
        "meteo_if_smoke", 
        "meteo_if_mist",
        "meteo_if_lightning", 
        "meteo_evapotranspiration_Monteith", 
        "meteo_radiation_UV", 
        "meteo_snow_height", 
        "meteo_snow_thickness_max", 
        "meteo_snow_thickness_6h"]

ajouts_drop_yael = ["meteo_altitude","meteo_temperature_min","meteo_temperature_max"]


columns_to_drop+=ajouts_drop_yael

In [4]:
# # Define the path to the dataset
# path_src_dataset = Path("./data/src/X_train_Hi5.csv")

# # Read only the column names without loading the data
# all_columns = pd.read_csv(path_src_dataset, nrows=0).columns.tolist()

# # Define columns to keep
# columns_to_keep = ['piezo_station_longitude', 'piezo_station_latitude', 'meteo_date', 'meteo_temperature_avg']

# # Determine columns to drop
# columns_to_drop = [col for col in all_columns if col not in columns_to_keep]

In [5]:
# Specification des colonnes  a garder 

##ANCIENNE VERSION
#all_columns = pd.read_csv(path_src_dataset, nrows=0).columns.tolist()
#columns_to_keep = [col for col in all_columns if col not in columns_to_drop] 


##NOUVELLE VERSION 
#Je précise ci dessous les colonnes nécessaires pour chacun des transformers, et en commentaire l'ordre dans lequel ce transformer sera utilisé
#Date
col_yass = ['meteo_date'] + ['prelev_volume_0', 'prelev_volume_1',
                             'prelev_volume_2', 'prelev_other_volume_sum']  # ORDRE = 4 (en dernier car il supprime les dates )
#Altitude
altitude_flo = ["piezo_station_altitude", "meteo_altitude"] # ORDRE 1
prelev_flo = ["prelev_volume_0", "prelev_usage_label_0", "prelev_volume_obtention_mode_label_0", "prelev_volume_1", "prelev_usage_label_1", "prelev_volume_obtention_mode_label_1", "prelev_volume_2", "prelev_usage_label_2", "prelev_volume_obtention_mode_label_2"]
col_flo = altitude_flo + prelev_flo
#Insee & rain "CleanFeatures"
cols_yael_input = ['insee_%_agri', 'meteo_rain_height', 'insee_pop_commune', 'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
cols_yael_need = ["piezo_station_department_code", "meteo_date"]

#Temperature
cols_lucien_need = ['piezo_station_department_code', 'piezo_measurement_date']
cols_lucien_input = ['meteo_temperature_avg','meteo_temperature_min','meteo__pressure_saturation_avg','meteo_temperature_max']
#Lat Long
cols_mat = ["distance_piezo_meteo",'piezo_station_longitude','piezo_station_latitude','meteo_latitude','meteo_longitude', "hydro_observation_result_elab"]


#Clean pizo
pizo_cols = ['piezo_station_investigation_depth', 'piezo_obtention_mode', 'piezo_status', 'piezo_qualification', 'piezo_measure_nature_code','piezo_station_department_code']
 
#target
target = "piezo_groundwater_level_category"

columns_to_keep = col_yass + cols_yael_input + cols_yael_need + col_flo + cols_lucien_need + cols_lucien_input + cols_mat + [target] + pizo_cols

columns_to_keep

['meteo_date',
 'insee_%_agri',
 'meteo_rain_height',
 'insee_pop_commune',
 'insee_med_living_level',
 'insee_%_ind',
 'insee_%_const',
 'piezo_station_department_code',
 'meteo_date',
 'piezo_station_altitude',
 'meteo_altitude',
 'prelev_volume_0',
 'prelev_usage_label_0',
 'prelev_volume_obtention_mode_label_0',
 'prelev_volume_1',
 'prelev_usage_label_1',
 'prelev_volume_obtention_mode_label_1',
 'prelev_volume_2',
 'prelev_usage_label_2',
 'prelev_volume_obtention_mode_label_2',
 'piezo_station_department_code',
 'piezo_measurement_date',
 'meteo_temperature_avg',
 'meteo_temperature_min',
 'meteo__pressure_saturation_avg',
 'meteo_temperature_max',
 'distance_piezo_meteo',
 'piezo_station_longitude',
 'piezo_station_latitude',
 'meteo_latitude',
 'meteo_longitude',
 'hydro_observation_result_elab',
 'piezo_groundwater_level_category',
 'piezo_station_investigation_depth',
 'piezo_obtention_mode',
 'piezo_status',
 'piezo_qualification',
 'piezo_measure_nature_code',
 'piezo_stat

In [6]:
path_src_dataset = Path("./data/src/X_train_Hi5.csv")


# Out folders
out_folder_dataset = Path("./data/cleaned")
out_folder_dataset.mkdir(parents=True, exist_ok=True) # Create the folder if it doesn't exist
out_folder_config = Path("./data/processed/pipelines")
out_folder_config.mkdir(parents=True, exist_ok=True)

# Load the CSV file with only the relevant columns
df = pd.read_csv(path_src_dataset, usecols=columns_to_keep ,nrows=10e4) #  SI on veut charger moins de lignes : ajouter --> ,nrows=10e4)



X = df.drop(columns=target)

#Mapping du target 
mapping = {'Very Low': 0, 'Low': 1, 'Average': 2, 'High': 3, 'Very High': 4}
y = df[target].map(mapping)

#Test-val split 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Apply the transformers selected
processing_pipeline = Pipeline(steps=[
("DropNaRate", DropNaRate(0.7)),
("Prelevement", Prelev(columns=col_flo,usage_label_max_categories=4,mode_label_max_categories=4,scale=1)),
("CleanFeatures", CleanFeatures(cols_yael_input)),
("Altitude", AltitudeTrans(columns=["piezo_station_altitude", "meteo_altitude"])),
('LatLong',CleanLatLon()),
('CleanTemp',CleanTemp()), 
('PrelevVol', Prelev()),
('Temp',TemperaturePressionTrans(columns=cols_lucien_input)),
('CleanHydro', CleanHydro()),
('CleanPizo',  CleanPizo(pizo_cols)),
('Dates',DateTransformer()),
('DropCols',DropCols(columns_to_drop) )

# ... Add others transformations
])

/var/folders/1h/b1s2w7bs0rd5fvjqfq1pbky80000gn/T/ipykernel_24266/750384322.py:11: DtypeWarning: Columns (110,111,117,118,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_src_dataset, usecols=columns_to_keep) #  SI on veut charger moins de lignes : ajouter --> ,nrows=10e4)


In [7]:
print("Pipeline ongoing...")
processed_X_train = processing_pipeline.fit_transform(X_train)
processed_X_val = processing_pipeline.transform(X_val)
processed_X_train.describe()

# Save the processed data to CSV
# processed_X_train.to_csv(out_folder_dataset / "X_train.csv", index=False)
# processed_X_val.to_csv(out_folder_dataset / "X_val.csv", index=False)
# y_train.to_csv(out_folder_dataset / "y_train.csv", index=False)
# y_val.to_csv(out_folder_dataset / "y_val.csv", index=False)

# print("Data converted to csv")

Pipeline ongoing...
>> (Info) Droped columns : []
>> (Info - Prelev) 'prelev_volume_0' has been filledna with mean = 1438270324.4788396
>> (Info - Prelev) 'prelev_usage_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_0' has been one-hot-encoded in 4 features
>> (Info) Recuperations des moyennes des données INSEE par department
>> (Info) Infos medianes Insee recupérees
>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (Info) Calculating means for numerical features and preparing for one-hot encoding.
>> (Info) Fitting completed: Means, modes, and one-hot encoders prepared.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-h

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,meteo_date,meteo_rain_height,meteo_temperature_avg,meteo__pressure_saturation_avg,distance_piezo_meteo,hydro_observation_result_elab,...,piezo_status_Donnée contrôlée niveau 2,piezo_status_Donnée interprétée,piezo_qualification_Correcte,piezo_qualification_Incertaine,piezo_qualification_Incorrecte,piezo_qualification_Non qualifié,piezo_measure_nature_code_0,piezo_measure_nature_code_D,piezo_measure_nature_code_I,piezo_measure_nature_code_N
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,8.000000e+04,...,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,90.072001,149.880498,2.221560,46.422956,0.924389,1.925792,37.687337,75.091392,0.950250,4.960411e+04,...,0.781400,0.021750,0.982513,0.003650,0.003200,0.010637,0.590437,0.005000,0.039012,0.365550
std,240.363245,180.345925,2.724138,2.174254,0.067875,4.578783,57.633061,68.844381,0.217429,1.923726e+05,...,0.413299,0.145867,0.131080,0.060305,0.056478,0.102589,0.491756,0.070534,0.193626,0.481587
min,0.000000,0.720000,-4.657129,41.399732,0.772456,0.000000,-8.300000,0.500000,0.000000,1.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.500000,41.900000,-0.105584,44.499602,0.869764,0.000000,5.600000,8.400000,1.000000,1.072000e+03,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,41.000000,108.000000,1.897576,46.393226,0.941397,0.200000,8.700000,13.200000,1.000000,3.243000e+03,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,100.000000,185.000000,4.216516,48.214196,0.985240,2.000000,13.000000,146.429890,1.000000,1.517300e+04,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,5233.400000,2150.000000,9.519890,50.971087,1.000000,189.000000,146.429890,146.429890,1.000000,3.235001e+06,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
processed_X_train.columns

Index(['piezo_station_investigation_depth', 'piezo_station_altitude',
       'piezo_station_longitude', 'piezo_station_latitude', 'meteo_date',
       'meteo_rain_height', 'meteo_temperature_avg',
       'meteo__pressure_saturation_avg', 'distance_piezo_meteo',
       'hydro_observation_result_elab', 'insee_%_agri', 'insee_pop_commune',
       'insee_med_living_level', 'insee_%_ind', 'insee_%_const',
       'hydro_observation_log',
       'piezo_obtention_mode_Mode d'obtention inconnu',
       'piezo_obtention_mode_Valeur mesurée',
       'piezo_obtention_mode_Valeur reconstituée', 'piezo_status_Donnée brute',
       'piezo_status_Donnée contrôlée niveau 1',
       'piezo_status_Donnée contrôlée niveau 2',
       'piezo_status_Donnée interprétée', 'piezo_qualification_Correcte',
       'piezo_qualification_Incertaine', 'piezo_qualification_Incorrecte',
       'piezo_qualification_Non qualifié', 'piezo_measure_nature_code_0',
       'piezo_measure_nature_code_D', 'piezo_measure_nature_c

In [9]:
a

NameError: name 'a' is not defined

# Quick KNN

In [ ]:
processed_X_train.head()

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,meteo_date,meteo_rain_height,meteo_temperature_avg,meteo__pressure_saturation_avg,distance_piezo_meteo,hydro_observation_result_elab,...,piezo_status_Donnée interprétée,piezo_qualification_Correcte,piezo_qualification_Incertaine,piezo_qualification_Incorrecte,piezo_qualification_Non qualifié,piezo_measure_nature_code_146.42989,piezo_measure_nature_code_0,piezo_measure_nature_code_D,piezo_measure_nature_code_I,piezo_measure_nature_code_N
0,102.0,51.74,-0.682180,44.744479,0.869764,3.6,13.70000,14.60000,1.0,553.0,...,0,1,0,0,0,0,1,0,0,0
1,166.0,14.54,2.985469,42.584073,0.947060,88.0,5.80000,6.20000,1.0,3274.0,...,0,1,0,0,0,0,0,0,0,1
2,140.0,55.00,-0.499435,45.778934,0.952442,0.0,146.42989,146.42989,1.0,86602.0,...,0,1,0,0,0,0,1,0,0,0
3,372.0,111.32,-0.399286,43.652475,0.996303,0.0,4.60000,8.10000,1.0,228.0,...,0,1,0,0,0,0,1,0,0,0
4,36.0,84.91,3.379233,43.539801,0.804187,0.0,9.50000,9.30000,1.0,27221.0,...,1,1,0,0,0,0,0,0,1,0


In [ ]:
#quick KNN
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

global_pipeline = Pipeline([
    ('processing', processing_pipeline),
    ('StandardScaler', PartialStandardScaler(columns='all')),
    ('estimator', RandomForestClassifier())
])

In [14]:
global_pipeline.fit(X_train, y_train)

>> (Info) Droped columns : []
>> (Info) Recuperations des moyennes des données INSEE par department
>> (Info) Infos medianes Insee recupérees
>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (Info) Calculating means for numerical features and preparing for one-hot encoding.
>> (Info) Fitting completed: Means, modes, and one-hot encoders prepared.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding applied to piezo_qualification with missing values filled.
>> (Info) One-hot encoding applied to piezo_measure_nature_code with missing values filled.
>> (Info) Data transformation completed.
>> (INFO - DropCols) columns ['piezo_station_department_name', 'piezo_station_up

Pipeline(steps=[('processing',
                 Pipeline(steps=[('DropNaRate', DropNaRate(rate=0.7)),
                                 ('CleanFeatures',
                                  CleanFeatures(cols_to_handle=['insee_%_agri',
                                                                'meteo_rain_height',
                                                                'insee_pop_commune',
                                                                'insee_med_living_level',
                                                                'insee_%_ind',
                                                                'insee_%_const'])),
                                 ('Altitude',
                                  AltitudeTrans(columns=['piezo_station_altitude',
                                                         'meteo_altitude'])),
                                 ('LatLong',
                                  CleanLatLon(dist_...
                                                'piezo_status_Donnée contrôlée '
                                                'niveau 2',
                                                'piezo_status_Donnée '
                                                'interprétée',
                                                'piezo_qualification_Correcte',
                                                'piezo_qualification_Incertaine',
                                                'piezo_qualification_Incorrecte',
                                                'piezo_qualification_Non '
                                                'qualifié',
                                                'piezo_measure_nature_code_146.42989',
                                                'piezo_measure_nature_code_0',
                                                'piezo_measure_nature_code_D', ...])),
                ('estimator', KNeighborsClassifier(n_neighbors=25))])

In [15]:
y_val_pred = global_pipeline.predict(X_val)

# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))

>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding applied to piezo_qualification with missing values filled.
>> (Info) One-hot encoding applied to piezo_measure_nature_code with missing values filled.
>> (Info) Data transformation completed.
>> (INFO - DropCols) columns ['piezo_station_department_name', 'piezo_station_update_date', 'piezo_station_commune_code_insee', 'piezo_station_pe_label', 'piezo_station_bdlisa_codes', 'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code', 'piezo_measurement_date', 'piezo_producer_name', 'piezo_measure_nature_code', 'meteo_name', 'meteo_id', 'meteo_latitude', 'meteo_lo

### Save Pipeline

In [16]:
save = True
if save:
        
    pipeline_name = "pipeline_randomforest_1st"

    # Writing to sample.json
    with open(out_folder_config / Path(pipeline_name + ".pkl"), "wb") as file:
        pickle.dump(global_pipeline, file)